In [1]:
import pandas as pd
month_09 = pd.read_parquet('./data/month_09.parquet')
station = pd.read_csv('./data/station.csv')

In [2]:
month_09

,rental_id,rent_datetime,return_datetime,distance,user_type,gender,birth_year,rent_station_id,return_station_id,bike_type
0,2016090100000001,2016-09-01 00:00:10,2016-09-01 00:25:41,4820.0,USR_001,M,1987.0,ST-400,ST-189,LCD자전거
1,2016090100000002,2016-09-01 00:00:10,2016-09-01 00:08:53,1540.0,USR_001,M,1997.0,ST-208,ST-8,LCD자전거
2,2016090100000003,2016-09-01 00:00:10,2016-09-01 00:18:22,0.0,USR_001,M,1987.0,ST-32,ST-223,LCD자전거
3,2016090100000004,2016-09-01 00:00:13,2016-09-01 00:11:22,2520.0,USR_001,M,1984.0,ST-28,ST-18,LCD자전거
4,2016090100000005,2016-09-01 00:00:22,2016-09-01 00:15:34,0.0,USR_001,M,1988.0,ST-253,ST-253,LCD자전거
...,...,...,...,...,...,...,...,...,...,...
335327,2016093023000620,2016-09-30 23:59:18,2016-10-01 01:19:13,10730.0,USR_001,M,1986.0,ST-11,ST-195,LCD자전거
335328,2016093023000621,2016-09-30 23:59:24,2016-10-01 00:56:55,8340.0,USR_001,F,1988.0,ST-330,ST-442,LCD자전거
335329,2016093023000622,2016-09-30 23:59:24,2016-10-01 00:12:25,0.0,USR_001,F,1994.0,ST-249,ST-108,LCD자전거
335330,2016093023000623,2016-09-30 23:59:48,2016-10-01 00:03:50,860.0,USR_001,M,1980.0,ST-38,ST-36,LCD자전거


In [3]:
station

,station_id,station_name,district,latitude,longitude
0,ST-10,서교동 사거리,마포구,37.55275,126.91862
1,ST-100,더샵스타시티 C동 앞,광진구,37.53667,127.07359
2,ST-1000,서부식자재마트 건너편,양천구,37.51038,126.86680
3,ST-1002,서울시 도로환경관리센터,양천구,37.52990,126.87654
4,ST-1003,신월중학교,양천구,37.53955,126.82830
...,...,...,...,...,...
3197,ST-993,경서농협 버스정류장(우리은행신정동지점방향),양천구,37.52151,126.85738
3198,ST-994,SBS방송국,양천구,37.52916,126.87275
3199,ST-995,계남공원 입구 주출입구 좌측,양천구,37.51060,126.85732
3200,ST-996,양강중학교앞 교차로,양천구,37.52433,126.85055


## duckDB
전체 용량 50GB -> parquet으로 압축했지만 DataFrame으로 불러오기엔 메모리 턱없이 부족함.  
-> duckdb로 출발-도착 groupby집계후 계산 결과를 station.csv와 합쳐 분석에 이용하도록 한다.

**왕복 무시 & 집계**

In [4]:
import duckdb
import pandas as pd
from dotenv import load_dotenv
import os

# 환경변수 로드
load_dotenv()

def setup_duckdb_s3():
    """
    DuckDB S3 연결 설정
    """
    con = duckdb.connect()
    
    # S3 확장 설치 및 로드
    con.execute("INSTALL httpfs;")
    con.execute("LOAD httpfs;")
    
    # AWS 자격증명 설정
    access_key = os.getenv('AWS_ACCESS_KEY_ID')
    secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    con.execute(f"SET s3_access_key_id='{access_key}';")
    con.execute(f"SET s3_secret_access_key='{secret_key}';")
    con.execute("SET s3_region='ap-northeast-2';")  # 서울 리전
    
    return con


def aggregate_od_pairs(con, bucket_name, year='2016'):
    """
    S3 parquet에서 OD pair 집계
    
    Returns:
    pd.DataFrame: columns=[station_a, station_b, count]
    """
    print("Aggregating OD pairs...")
    
    query = f"""
    SELECT 
        CASE 
            WHEN rent_station_id < return_station_id 
            THEN rent_station_id 
            ELSE return_station_id 
        END as station_a,
        CASE 
            WHEN rent_station_id < return_station_id 
            THEN return_station_id 
            ELSE rent_station_id 
        END as station_b,
        COUNT(*) as trip_count
    FROM read_parquet('s3://{bucket_name}/rent/{year}/*.parquet')
    WHERE rent_station_id != return_station_id  -- 순환 제외
    GROUP BY station_a, station_b
    ORDER BY trip_count DESC
    """
    
    result = con.execute(query).df()
    
    print(f"Total unique OD pairs: {len(result):,}")
    print(f"Total trips: {result['trip_count'].sum():,}")
    
    return result


# 실행
con = setup_duckdb_s3()
od_pairs = aggregate_od_pairs(con, 'yn-project-seoul-bike', year='2016')

print("\nTop 10 routes:")
print(od_pairs.head(10))

Aggregating OD pairs...
Total unique OD pairs: 36,430
Total trips: 1,244,075

Top 10 routes:
  station_a station_b  trip_count
0     ST-14     ST-18        4529
1     ST-18    ST-341        3428
2    ST-391    ST-406        2885
3     ST-98     ST-99        2756
4     ST-47     ST-73        2705
5     ST-18      ST-9        2626
6     ST-91     ST-95        2608
7     ST-68     ST-71        2586
8     ST-86     ST-95        2541
9     ST-27     ST-28        2515


In [5]:
od_pairs

,station_a,station_b,trip_count
0,ST-14,ST-18,4529
1,ST-18,ST-341,3428
2,ST-391,ST-406,2885
3,ST-98,ST-99,2756
4,ST-47,ST-73,2705
...,...,...,...
36425,ST-204,ST-76,1
36426,ST-10,ST-107,1
36427,ST-182,ST-199,1
36428,ST-443,ST-470,1


**Station 정보 합치기**

In [6]:
def load_station_info(con, bucket_name):
    """
    S3에서 station 정보 로드
    """
    query = f"""
    SELECT *
    FROM read_csv_auto('s3://{bucket_name}/station/station.csv')
    """
    
    station = con.execute(query).df()
    print(f"Total stations: {len(station):,}")
    
    return station


def merge_station_info(od_pairs, station):
    """
    OD pair에 station 위경도 정보 합치기
    
    Returns:
    pd.DataFrame: [station_a, station_b, trip_count, 
                   lat_a, lon_a, lat_b, lon_b, ...]
    """
    # station_a 정보 합치기
    result = od_pairs.merge(
        station[['station_id', 'latitude', 'longitude', 'station_name', 'district']],
        left_on='station_a',
        right_on='station_id',
        how='left'
    ).rename(columns={
        'latitude': 'lat_a',
        'longitude': 'lon_a',
        'station_name': 'name_a',
        'district': 'district_a'
    }).drop('station_id', axis=1)
    
    # station_b 정보 합치기
    result = result.merge(
        station[['station_id', 'latitude', 'longitude', 'station_name', 'district']],
        left_on='station_b',
        right_on='station_id',
        how='left'
    ).rename(columns={
        'latitude': 'lat_b',
        'longitude': 'lon_b',
        'station_name': 'name_b',
        'district': 'district_b'
    }).drop('station_id', axis=1)
    
    print(f"Merged {len(result):,} OD pairs with station info")
    
    return result


# 실행
station = load_station_info(con, 'yn-project-seoul-bike')
od_with_location = merge_station_info(od_pairs, station)

print(od_with_location.head())

Total stations: 3,202
Merged 36,430 OD pairs with station info
  station_a station_b  trip_count     lat_a      lon_a        name_a  \
0     ST-14     ST-18        4529  37.56454  126.92707    일진아이윌아파트 옆   
1     ST-18    ST-341        3428  37.55744  126.92382  홍대입구역 2번출구 앞   
2    ST-391    ST-406        2885  37.58370  127.05386  서울시립대 정문 앞 B   
3     ST-98     ST-99        2756  37.53731  127.07035    광진문화예술회관 앞   
4     ST-47     ST-73        2705  37.52882  126.92453        국민일보 앞   

  district_a     lat_b      lon_b              name_b district_b  
0       서대문구  37.55744  126.92382        홍대입구역 2번출구 앞        마포구  
1        마포구  37.56539  126.91933       하늘채코오롱아파트 건너편        마포구  
2       동대문구  37.58504  127.06027            서울시립대 후문       동대문구  
3        광진구  37.53186  127.06719  자양(뚝섬한강공원)역 1번출구 앞        광진구  
4       영등포구  37.52716  126.93190        여의나루역 1번출구 앞       영등포구  


In [7]:
import osmnx as ox
import networkx as nx
import pickle
from tqdm import tqdm

def get_seoul_bike_network():
    """
    서울 자전거 도로 네트워크 다운로드
    (한번만 실행, 저장해두고 재사용)
    """
    print("Downloading Seoul bike network...")
    
    # 서울시 전체 자전거 네트워크
    G = ox.graph_from_place(
        'Seoul, South Korea',
        network_type='bike',
        simplify=True
    )
    
    # 저장
    ox.save_graphml(G, 'seoul_bike_network.graphml')
    print(f"Network saved: {len(G.nodes)} nodes, {len(G.edges)} edges")
    
    return G


def calculate_route(G, lat_a, lon_a, lat_b, lon_b):
    """
    두 지점 간 자전거 경로 계산
    
    Returns:
    list: 경로상의 노드 리스트 or None
    """
    try:
        # 가장 가까운 네트워크 노드 찾기
        orig_node = ox.nearest_nodes(G, lon_a, lat_a)
        dest_node = ox.nearest_nodes(G, lon_b, lat_b)
        
        # 최단 경로 계산
        route = nx.shortest_path(G, orig_node, dest_node, weight='length')
        
        return route
    
    except Exception as e:
        # 경로 못찾으면 None
        return None


# 네트워크 로드 (이미 있으면 로드, 없으면 다운로드)
try:
    G = ox.load_graphml('seoul_bike_network.graphml')
    print(f"Loaded existing network: {len(G.nodes)} nodes")
except:
    G = get_seoul_bike_network()

Loaded existing network: 118856 nodes


In [8]:
def add_bike_priority_weights(G):
    """
    자전거 도로 우선순위를 가중치에 반영
    
    우선순위: 자전거전용도로 > 자전거도로 > 차도
    """
    print("Adding bike priority weights...")
    
    for u, v, data in G.edges(data=True):
        base_length = data.get('length', 100)
        
        # highway 타입 확인
        highway = data.get('highway', '')
        if isinstance(highway, list):
            highway = highway[0]
        
        # 자전거 도로 타입
        cycleway = data.get('cycleway', '')
        
        # 가중치 배율 설정
        if cycleway in ['track', 'lane']:  # 자전거 전용/차선
            penalty = 0.5  # 50% 선호
        elif 'cycleway' in str(highway) or highway == 'path':
            penalty = 0.7
        elif highway in ['residential', 'living_street']:  # 주거지역
            penalty = 0.9
        elif highway in ['primary', 'secondary', 'tertiary']:  # 큰 차도
            penalty = 1.5  # 회피
        elif highway in ['trunk', 'motorway']:  # 고속도로
            penalty = 3.0  # 최대 회피
        else:  # 일반 도로
            penalty = 1.0
        
        # 조정된 가중치 저장
        data['bike_weight'] = base_length * penalty
    
    print("Bike priority weights applied")
    return G


def calculate_route_with_priority(G, lat_a, lon_a, lat_b, lon_b):
    """
    우선순위 반영한 경로 계산
    
    Returns:
    dict: {'nodes': [...], 'length': float} or None
    """
    try:
        orig_node = ox.nearest_nodes(G, lon_a, lat_a)
        dest_node = ox.nearest_nodes(G, lon_b, lat_b)
        
        # 우선순위 반영 경로
        route_nodes = nx.shortest_path(
            G, orig_node, dest_node, 
            weight='bike_weight'
        )
        
        # 실제 거리 계산
        length = sum(
            G[route_nodes[i]][route_nodes[i+1]][0].get('length', 0)
            for i in range(len(route_nodes)-1)
        )
        
        return {
            'nodes': route_nodes,
            'length': length
        }
    
    except Exception as e:
        return None


# 가중치 적용
G = add_bike_priority_weights(G)

# 테스트
print("\n=== Testing route calculation ===")
test_row = od_with_location.iloc[0]
print(f"Testing: {test_row['name_a']} → {test_row['name_b']}")

route = calculate_route_with_priority(
    G, 
    test_row['lat_a'], test_row['lon_a'],
    test_row['lat_b'], test_row['lon_b']
)

if route:
    print(f"✓ Route found!")
    print(f"  Nodes: {len(route['nodes'])}")
    print(f"  Length: {route['length']:.0f}m")
else:
    print("✗ Route not found")

Adding bike priority weights...
Bike priority weights applied

=== Testing route calculation ===
Testing: 일진아이윌아파트 옆 → 홍대입구역 2번출구 앞
✓ Route found!
  Nodes: 40
  Length: 1154m
